In [2]:
import numpy as np
import gurobipy as gp
from gurobipy import GRB

In [3]:
gate = np.load('gate_8.npy', allow_pickle=True).item()
flight = np.load('flight_20.npy', allow_pickle=True).item()

In [4]:
gate

{'gate_type': ['Domestic',
  'Domestic',
  'Domestic',
  'Domestic',
  'International',
  'International',
  'International',
  'International',
  'Apron'],
 'distance_gate_entrance': array([ 9.,  7.,  5.,  3.,  3.,  5.,  7.,  9., 33.]),
 'distance_between_gate': array([[ 0.,  2.,  4.,  6.,  8., 10., 12., 14., 30.],
        [ 2.,  0.,  2.,  4.,  6.,  8., 10., 12., 30.],
        [ 4.,  2.,  0.,  2.,  4.,  6.,  8., 10., 30.],
        [ 6.,  4.,  2.,  0.,  2.,  4.,  6.,  8., 30.],
        [ 8.,  6.,  4.,  2.,  0.,  2.,  4.,  6., 30.],
        [10.,  8.,  6.,  4.,  2.,  0.,  2.,  4., 30.],
        [12., 10.,  8.,  6.,  4.,  2.,  0.,  2., 30.],
        [14., 12., 10.,  8.,  6.,  4.,  2.,  0., 30.],
        [30., 30., 30., 30., 30., 30., 30., 30.,  0.]])}

In [5]:
flight

{'flight_type': ['International',
  'International',
  'International',
  'International',
  'Domestic',
  'Domestic',
  'Domestic',
  'Domestic',
  'Domestic',
  'Domestic',
  'International',
  'Domestic',
  'Domestic',
  'International',
  'International',
  'International',
  'Domestic',
  'International',
  'International',
  'Domestic'],
 'arrival_time': array([  8,  10,  12,  25,  59,  65, 201, 210, 213, 217, 224, 248, 303,
        323, 329, 395, 406, 422, 429, 468]),
 'departure_time': array([ 62,  77,  84,  82, 116, 120, 252, 286, 263, 286, 285, 307, 357,
        373, 382, 445, 480, 488, 499, 530]),
 'non_transfer_passengers': array([145, 132, 144, 126, 133, 101, 141, 131,  94, 138,  77, 131, 112,
        140, 124, 120, 111,  85, 118,  85]),
 'transfer_passengers': array([[0, 6, 6, 6, 0, 4, 6, 6, 0, 4, 4, 4, 5, 3, 0, 2, 3, 0, 5, 3],
        [0, 0, 3, 2, 4, 3, 6, 0, 3, 1, 5, 5, 3, 2, 5, 2, 2, 1, 0, 1],
        [0, 0, 0, 3, 5, 3, 6, 1, 6, 4, 2, 0, 2, 5, 0, 1, 5, 1, 5, 6],
      

In [6]:
gate_type = gate['gate_type']
gateNum = len(gate_type) - 1
flight_type = flight['flight_type']
flightNum = len(flight_type)

In [7]:
a_time = flight['arrival_time']
d_time = flight['departure_time']

In [8]:
n_d = flight_type.count('Domestic')
n_d_g = gate_type.count('Domestic')

In [9]:
nw_d = np.zeros([flightNum + 1,flightNum +2])

In [10]:
for i in range(1,a_time.shape[0] + 1):
    if flight_type[i-1] == 'Domestic':
        nw_d[i][-1] =1
    for j in range( i,a_time.shape[0] + 1):
        if flight_type[j-1] == 'Domestic':
            nw_d[0][j] = 1
        if (a_time[j-1] > d_time[i-1]) and (flight_type [i-1] == 'Domestic') and (flight_type [j-1] == 'Domestic') :
            nw_d[i][j] = 1
            
#[0][a_time.shape[0]] = 1
#nw_d[-1][-1] = 1

In [11]:
nw_d[:,:]

array([[0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 1., 0., 1., 1., 0., 0.,
        0., 1., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 0., 1., 1., 0., 0.,
        0., 1., 0., 0., 1., 1.],
       [0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 0., 1., 1., 0., 0.,
        0., 1., 0., 0., 1., 1.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
        0., 1., 0., 0., 1., 1.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
        0., 1., 0., 0., 1., 1.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,

In [11]:
M = flightNum
m = gp.Model()
z_d = m.addMVar(shape = (M + 1, M + 2),vtype='B' ,name = 'z_d')

z_sum = np.sum([np.sum([z_d[i,j] for j in range(i+1, M+1)]) for i in range(M+1)])
m.setObjective(z_sum,GRB.MAXIMIZE)

for i in range(0,M + 1):
    for j in range(0, M + 2):
        m.addConstr(z_d[i,j] <= nw_d[i,j]) #Zij must be valid

m.addConstr(z_d[0].sum() <= n_d_g)   
m.addConstr(z_d[:,M+1].sum() <= n_d_g)
    
for i in range(1,M+1): 
    m.addConstr(z_d[:,i].sum() <= 1) #sum of each column <= 1
for j in range(1,M+1): 
    m.addConstr(z_d[j,:].sum() <= 1) #sum of each row <= 1

for i in range(1,M+1): 
    m.addConstr(z_d[:,i].sum() == z_d[i].sum())
    
m.optimize()

Set parameter Username
Academic license - for non-commercial use only - expires 2023-10-04
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 524 rows, 462 columns and 2185 nonzeros
Model fingerprint: 0xc5dddbe6
Variable types: 0 continuous, 462 integer (462 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+00]
Found heuristic solution: objective -0.0000000
Presolve removed 500 rows and 410 columns
Presolve time: 0.06s
Presolved: 24 rows, 52 columns, 148 nonzeros
Variable types: 0 continuous, 52 integer (52 binary)
Found heuristic solution: objective 9.0000000

Root relaxation: cutoff, 27 iterations, 0.01 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node

In [12]:
print('Total flights assigned to gates:'  + str(int(m.ObjVal)))
#print('Number of flights assigned to Apron: ' + str(int(M - m.ObjVal)))

print()
print("   ", end = '')
for i in range(M+2):
    print(str(i), end = '  ')
print()
for i in range(M+1):
    print(str(i)+': ',end='')
    for j in range(M+2):
        print(int(m.getVars()[i*(M+2)+j].x), end = '  ')
    print()

Total flights assigned to gates:9

   0  1  2  3  4  5  6  7  8  9  10  11  12  13  14  15  16  17  18  19  20  21  
0: 0  0  0  0  0  1  1  1  1  0  0  0  0  0  0  0  0  0  0  0  0  0  
1: 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  
2: 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  
3: 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  
4: 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  
5: 0  0  0  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0  
6: 0  0  0  0  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0  0  
7: 0  0  0  0  0  0  0  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0  
8: 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  1  0  0  0  0  
9: 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  1  0  
10: 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  1  
11: 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  
12: 0  0  0  0  0  0  0  0  0  0  0  0  0